In [1]:
import numpy as np
import tvm
from tvm import te
from tvm import relay
from tvm.relay.testing import run_as_python
from tvm.relay.prelude import Prelude
from tvm.runtime.container import ADT
from tvm.relay.backend.interpreter import RefValue, ConstructorValue

In [2]:
def seq(*exprs):
    """helper: uses a dummy let binding to sequence a list
       of expressions: expr1; expr2; expr3, etc.
    """
    ret = exprs[0]
    for expr in exprs[1:]:
        ret = relay.Let(relay.var("_"), ret, expr)
    return ret

def init_box_adt(mod):
    """creates a dummy ADT for testing"""
    box = relay.GlobalTypeVar("box")
    a = relay.TypeVar("a")
    box_ctor = relay.Constructor("box", [a], box)
    mod[box] = relay.TypeData(box, [a], [box_ctor])
    return (box, box_ctor)

# assert that the candidate is a NDArray with value val
def assert_tensor_value(candidate, val):
    assert isinstance(candidate, tvm.nd.NDArray)
    assert np.array_equal(candidate.numpy(), np.array(val))


# assert that the candidate is an ADT with the indicated number of fields
def assert_adt_len(candidate, fields):
    assert isinstance(candidate, ADT)
    assert len(candidate) == fields


# assert that the candidate is a ConstructorValue with the approrpaite constructor
# and number of fields
def assert_constructor_value(candidate, constructor, fields):
    assert isinstance(candidate, ConstructorValue)
    assert candidate.tag == constructor.tag
    assert len(candidate.fields) == fields

In [3]:
empty = relay.Tuple([])
tup_val = run_as_python(empty)
assert_adt_len(tup_val, 0)

In [4]:
scalar = relay.const(1)
tensor_val = run_as_python(scalar)
assert_tensor_value(tensor_val, 1)

In [5]:
type(tup_val)

tvm.runtime.container.ADT

In [6]:
import numpy as np
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T